In [80]:
import numpy as np 
import pandas as pd
import os

# 获取当前文件夹下的所有合同文件
def getCurrentFiles():
    odd_file_list = []
    for root, dirs, files in os.walk("."):
        if '客户购销合同' in root:
            for name in files:
                if os.path.splitext(name)[1] in (['.xls', '.xlsx']):
                    odd_file_list.append(os.path.join(root, name))
    return odd_file_list

# 处理所有合同文件
def oddDeal():
    odd_file_list = getCurrentFiles()
    list_df = []
    for item in odd_file_list:
        # 读取文件
        df = pd.read_excel(item).fillna('--')
        # 获取合同单号和制单时间
        odd_company_name = list(df[df['购销合同'].astype(str).str.contains('需方')].T.iloc[1].values)
        odd_numbers = list(df[df.values == '合同单号：'].T.values[-1])
        odd_date =list(df[df.values == '制单时间：'].T.values[-1])
        # 获取订单部分
        dict_columns = dict(zip(list(df.columns), list(df[df['购销合同'] == '序 号'].values[0])))
        df_deal_1 = df[df[df['购销合同'] == '序 号'].index[0] + 1: df[df['购销合同'] == '合计：'].index[0]].rename(columns=dict_columns).reset_index()
        # 合同单号,和制单时间的series
        odd_company_name_series = pd.Series(len(df_deal_1) * odd_company_name)
        odd_numbers_series = pd.Series(len(df_deal_1) * odd_numbers)
        odd_date_series = pd.Series(len(df_deal_1) * odd_date)
        res_df = pd.concat([df_deal_1, odd_numbers_series, odd_date_series, odd_company_name_series], axis=1).rename(columns={0: '合同单号', 1: '制单时间', 2: '公司名称'})
        list_df.append(res_df)
    res_df = pd.concat(list_df)
    return res_df.set_index('index')

res_df = oddDeal()
res_df.to_excel('/Users/monstar/Desktop/res.xlsx', index=False)